In [ ]:
# This code is part of a Qiskit project.
#
# (C) Copyright IBM 2025.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

"""
Entanglement Concentration
"""
from __future__ import annotations

import warnings

import numpy as np
from qiskit import QuantumCircuit

from ..utils import algorithm_globals


# pylint: disable=too-many-positional-arguments
def entanglement_concentration_data(
    training_size: int,
    test_size: int,
    n: int,
    mode: str = "easy",
    one_hot: bool = True,
    include_sample_total: bool = False,
    sampling_method: str = "grid",
    class_labels: list | None = None,
) -> (
    tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]
    | tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]
):
    r"""
    Generates a dataset that comprises of Quantum States with two different
    amounts of Concentration Of Entanglement (CE) and their corresponding class labels.
    These states are generated by the effect of two different pre-trained ansatz
    on fully seperable input states, training procedure and data is used in courtesy
    of L Schatzki et el. [1]. The datapoints can be fully separated using the SWAP
    test outlined in [2]. First, input states are randomly generated from a
    uniform distribution, using a sampling method determined by the ``sampling_method``
    argument. Next, based on the ``mode`` argument, two pre-trained circuits "A" and "B"
    are used for generating datapoints.

    CE can be interpreted as a measure of correlation between the different qubits.
    The ``mode`` argument supports two options. ``"easy"`` gives datapoints with CE
    values of ``0.0`` for first class and ``0.35`` for the second class. ``"hard"`` mode
    gives datapoints with CE of ``0.15`` for first class and ``0.25`` for second class.
    The user's classifiers can be benchmarked against these modes for their ability to
    separate the data into two classes based on CE.

    Current implementation supports only ``n`` values of 3, 4 and 8.

    ``sampling_method`` argument supports two options. ``"random_basis"`` and ``"standard_basis"``.
    Random basis generates Qubit states that are sampled randomly in the Bloch Sphere and takes the
    tensor product of all the qubits to build the input state. Standard basis generates only states
    that fall on the axes of the Bloch Sphere before taking the tensor product.

    **References:**

    [1] Havlíček V, Córcoles AD, Temme K, Harrow AW, Kandala A, Chow JM,
    Gambetta JM. *Supervised learning with quantum-enhanced feature spaces*.
    Nature. 2019 Mar;567(7747):209–212.
    `arXiv:1804.11326 <https://arxiv.org/abs/1804.11326>`_

    Parameters:
        training_size : Number of training samples per class.
        test_size :  Number of testing samples per class.
        n : Number of qubits (dimension of the feature space).
        gap : Separation gap :math:`\Delta` used when ``labelling_method="expectation"``.
            Default is 0.
        plot_data : If True, plots the sampled data (disabled automatically if
            ``n > 3``). Default is False.
        one_hot : If True, returns labels in one-hot format. Default is True.
        include_sample_total : If True, the function also returns the total number
            of accepted samples. Default is False.
        entanglement : Determines which second-order terms :math:`Z_i Z_j` appear in
            :math:`U_{\Phi(\vec{x})}`. The options are:

                * ``"linear"``: Includes terms :math:`Z_i Z_{i+1}`.
                * ``"circular"``: Includes ``"linear"`` terms plus :math:`Z_{n-1}Z_0`.
                * ``"full"``: Includes all pairwise terms :math:`Z_i Z_j`.

            Default is ``"full"``.
        sampling_method: The method used to generate uniform samples :math:`\vec{x}`.
            Choices are:

                * ``"grid"``: Chooses points from a uniform grid (supported only if ``n <= 3``)
                * ``"hypercube"``: Uses a variant of Latin Hypercube sampling for stratification
                * ``"sobol"``: Uses Sobol sequences

            Default is ``"grid"``.
        divisions : Must be specified if ``sampling_method="hypercube"``. This parameter
            determines the number of stratifications along each dimension. Recommended
            to be chosen close to ``training_size``.
        labelling_method : Method for assigning labels. The options are:

                * ``"expectation"``: Uses the expectation value of the observable.
                * ``"measurement"``: Performs a measurement in the computational basis.

            Default is ``"expectation"``.
        class_labels : Custom labels for the two classes when one-hot is not enabled.
            If not provided, the labels default to ``-1`` and ``+1``

    Returns:
        Tuple
        containing the following:

        * **training_features** : ``np.ndarray``
        * **training_labels** : ``np.ndarray``
        * **testing_features** : ``np.ndarray``
        * **testing_labels** : ``np.ndarray``

        If ``include_sample_total=True``, a fifth element (``np.ndarray``) is included
        that specifies the total number of accepted samples.
    """

In [ ]:
# This code is part of a Qiskit project.
#
# (C) Copyright IBM 2025.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

"""
Test Entanglement Concentration
"""

from test import QiskitMachineLearningTestCase

import unittest

# References

In [ ]:
import numpy as np
import os
from typing import List, Tuple
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector
from qiskit.circuit import ParameterVector
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

class HardwareEfficientDatasetGenerator:
    def __init__(self, base_path: str = "Hardware_Efficient"):
        self.base_path = base_path
        self.supported_qubits = [3, 4, 8]
        self.supported_depths_34 = list(range(1, 7))
        self.supported_depths_8 = [5, 6]
        self.supported_ce_34 = [0.05, 0.15, 0.25, 0.35]
        self.supported_ce_8_6 = [0.10, 0.25]
        self.supported_ce_8_5 = [0.15, 0.40, 0.45]

    def validate_params(self, qubits: int, depth: int, goal_ce: float) -> bool:
        # Validate inputs
        if qubits not in self.supported_qubits:
            raise ValueError(f"Unsupported qubit count: {qubits}. Choose from {self.supported_qubits}")

        if (qubits == 8):
            if depth not in self.supported_depths_8:
                raise ValueError(f"Unsupported depth: {depth}. Choose from {self.supported_depths_8}")
            if (depth == 6):
                if goal_ce not in self.supported_ce_8_6:
                    raise ValueError(f"Unsupported CE value: {goal_ce}. Choose from {self.supported_ce_8_6}")
            else:
                if goal_ce not in self.supported_ce_8_5:
                    raise ValueError(f"Unsupported CE value: {goal_ce}. Choose from {self.supported_ce_8_5}")
        else:
            if depth not in self.supported_depths_34:
                raise ValueError(f"Unsupported depth: {depth}. Choose from {self.supported_depths_34}")
            if goal_ce not in self.supported_ce_34:
                raise ValueError(f"Unsupported CE value: {goal_ce}. Choose from {self.supported_ce_34}")

        return True

    def _construct_filepath(self, qubits: int, depth: int, goal_ce: float) -> str:
        """
        Construct the full file path based on parameters and directory structure.

        Args:
            qubits: Number of qubits (3 or 4)
            depth: Circuit depth (1-6)
            goal_ce: Target concentratable entanglement (0.05, 0.15, 0.25, 0.35, 0.5)

        Returns:
            Full path to .npy weights file
        """

        self.validate_params(qubits, depth, goal_ce)

        # Validate goal_ce format
        ce_str = f"{int(goal_ce * 100):02d}"

        # Construct file path
        qubit_dir = f"{qubits}_Qubits"
        depth_dir = f"Depth_{depth}"
        filename = f"hwe_{qubits}q_ps_{ce_str}_{depth}_weights.npy"

        return os.path.join(self.base_path, qubit_dir, depth_dir, filename)

    def load_weights(self, qubits: int, depth: int, goal_ce: float) -> np.ndarray:
        """
        Load weights from the repository.

        Args:
            qubits: Number of qubits
            input_type: Type of input states
            goal_ce: Goal concentratable entanglement (0-1)
            depth: Circuit depth

        Returns:
            Weights as numpy array
        """
        file_path = self._construct_filepath(qubits, depth, goal_ce)

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Weights file not found at: {file_path}")

        return np.load(file_path)

    def hardware_efficient_ansatz(self, qc, params, qubits, depth):
        """Adds hardware-efficient ansatz to a QuantumCircuit"""
        param_idx = 0
        for d in range(depth):
            # Single-qubit rotations
            for q in range(qubits):
                qc.rx(params[param_idx], q)
                qc.ry(params[param_idx+1], q)
                qc.rz(params[param_idx+2], q)
                param_idx += 3

            # Entangling layer
            for q in range(qubits - 1):
                qc.cx(q, q+1)
            if qubits > 1:
                qc.cx(qubits-1, 0)

    def generate_states(self, qubits: int, depth: int, goal_ce: float,
                       num_samples: int = 100) -> Tuple[np.ndarray, np.ndarray]:
        weights = self.load_weights(qubits, depth, goal_ce)
        simulator = Aer.get_backend('aer_simulator')

        # Validate parameter count
        expected_params = depth * qubits * 3
        if len(weights.flatten()) != expected_params:
            raise ValueError(f"Parameter mismatch: {len(weights.flatten())} vs {expected_params}")

        input_states = []
        output_states = []

        for _ in range(num_samples):
            input_state = np.random.randint(0, 2**qubits)
            qc = QuantumCircuit(qubits)

            # Initialize state
            for q in range(qubits):
                if (input_state >> q) & 1:
                    qc.x(q)

            # Create parameterized circuit
            params = ParameterVector('θ', depth * qubits * 3)
            self.hardware_efficient_ansatz(qc, params, qubits, depth)

            # Bind loaded weights
            bound_qc = qc.assign_parameters(weights.flatten())

            # Simulate
            bound_qc.save_statevector()
            result = simulator.run(bound_qc).result()
            statevector = result.get_statevector()

            input_states.append(input_state)
            output_states.append(statevector.data)

        return np.array(input_states), np.array(output_states)

    def save_dataset(self, input_states: np.ndarray, output_states: np.ndarray,
                    save_dir: str, filename_prefix: str):
        """
        Save generated dataset to files.

        Args:
            input_states: Array of input computational basis states
            output_states: Array of output quantum states
            save_dir: Directory to save files
            filename_prefix: Prefix for output files
        """
        os.makedirs(save_dir, exist_ok=True)

        # Save input states
        np.save(os.path.join(save_dir, f"{filename_prefix}_inputs.npy"), input_states)

        # Save output states
        np.save(os.path.join(save_dir, f"{filename_prefix}_outputs.npy"), output_states)

    def create_classification_dataset(self,
                                    entanglement_levels: List[float] = [0.15, 0.35, 0.45],
                                    depths: int | list[int] = 3,
                                    num_samples: int = 3000,
                                    qubits: int = 4,
                                    random_state: int = 42) -> Tuple[np.ndarray, np.ndarray]:
        """
        Create a classified dataset of quantum states with different entanglement levels.

        Args:
            entanglement_levels: List of target CE values for different classes
            depths: Single depth for all classes or list of depths per class
            num_samples: Total number of samples in the dataset
            qubits: Number of qubits in generated states
            random_state: Seed for reproducible shuffling

        Returns:
            X: Array of quantum states (num_samples, 2**qubits)
            y: Array of class labels (num_samples,)

        Raises:
            ValueError: For invalid input combinations
        """
        # Input validation
        if isinstance(depths, int):
            depths = [depths] * len(entanglement_levels)
        elif len(depths) != len(entanglement_levels):
            raise ValueError("Length of depths must match entanglement_levels")

        for ce_level, depth in zip(entanglement_levels, depths):
            self.validate_params(qubits, depth, ce_level)

        # Calculate samples per class with remainder distribution
        samples_per_class, remainder = divmod(num_samples, len(entanglement_levels))
        class_samples = [samples_per_class + (1 if i < remainder else 0)
                        for i in range(len(entanglement_levels))]

        # Generate states for each class
        X, y = [], []
        for class_idx, (ce_level, depth, n_samples) in enumerate(zip(entanglement_levels,
                                                                depths,
                                                                class_samples)):
            print(f"Generating class {class_idx+1}/{len(entanglement_levels)}: "
                f"CE={ce_level}, depth={depth}, samples={n_samples}")

            _, states = self.generate_states(
                qubits=qubits,
                depth=depth,
                goal_ce=ce_level,
                num_samples=n_samples
            )

            X.append(states)
            y.append(np.full(n_samples, class_idx))

        # Combine and shuffle
        X = np.concatenate(X)
        y = np.concatenate(y)
        rng = np.random.default_rng(random_state)
        indices = rng.permutation(len(X))

        return X[indices], y[indices]

# Example usage in the __main__ block
if __name__ == "__main__":
    generator = HardwareEfficientDatasetGenerator(base_path="Hardware_Efficient")

    # Classification dataset example
    print("\nGenerating classification dataset:")
    qubits = 3
    X, y = generator.create_classification_dataset(
        entanglement_levels=[0.15, 0.35],
        depths=[3, 5],
        qubits=qubits,
        num_samples=1000
    )

    print("\nClassification dataset stats:")
    print(f"Total samples: {len(X)}")
    print(f"Class distribution: {np.bincount(y)}")
    print(f"State shape: {X[0].shape}") # 2**qubits
    print(f"First state:\n{X[0]}")
    print(f"Class label: {y[0]}")

    # Save classification dataset
    generator.save_dataset(
        input_states=X,  # Dummy inputs since we're using class labels
        output_states=y,
        save_dir="classification_datasets",
        filename_prefix=f"{qubits}q_ce_classification"
    )


In [ ]:
# This code is part of a Qiskit project.
#
# (C) Copyright IBM 2018, 2025.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

"""
Ad Hoc Dataset
"""
from __future__ import annotations

import warnings
import itertools as it

import numpy as np
from scipy.stats.qmc import Sobol
from qiskit.utils import optionals

from ..utils import algorithm_globals


# pylint: disable=too-many-positional-arguments
def ad_hoc_data(
    training_size: int,
    test_size: int,
    n: int,
    gap: int = 0,
    plot_data: bool = False,
    one_hot: bool = True,
    include_sample_total: bool = False,
    entanglement: str = "full",
    sampling_method: str = "grid",
    divisions: int = 0,
    labelling_method: str = "expectation",
    class_labels: list | None = None,
) -> (
    tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]
    | tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]
):
    r"""
    Generates a dataset that can be fully separated by
    :class:`~qiskit.circuit.library.ZZFeatureMap` according to the procedure
    outlined in [1]. First, vectors :math:`\vec{x} \in (0, 2\pi]^{n}` are generated from a
    uniform distribution, using a sampling method determined by the ``sampling_method``
    argument. Next, a feature map is applied:

    .. math::
       |\Phi(\vec{x})\rangle
       = U_{\Phi(\vec{x})} \, H^{\otimes n} \,
         U_{\Phi(\vec{x})} \, H^{\otimes n} \, |0^{\otimes n}\rangle

    where

    .. math::
       U_{\Phi(\vec{x})}
       = \exp\Bigl(i \sum_{S \subseteq [n]} \phi_S(\vec{x}) \prod_{i \in S} Z_i\Bigr),

    and

    .. math::
        \begin{cases}\phi_{\{i, j\}} = (\pi - x_i)(\pi - x_j) \\
        \phi_{\{i\}} = x_i \end{cases}

    The choice of second-order terms :math:`Z_i Z_j` in the above summation depends
    on the ``entanglement`` argument (``"linear"``, ``"circular"``, or
    ``"full"``). See arguments for more information.

    An observable is then defined as

    .. math::
       O = V^\dagger \bigl(\prod_i Z_i\bigr) V

    where :math:`V` is a randomly generated unitary matrix. Depending on the
    ``labelling_method``, if ``"expectation"`` is used, the expectation value
    :math:`\langle \Phi(\vec{x})| O |\Phi(\vec{x})\rangle` is compared to the
    gap parameter :math:`\Delta` (from ``gap``) to assign :math:`\pm 1` labels.
    if ``"measurement"`` is used, a simple measurement in the computational
    basis is performed to assign labels.

    **References:**

    [1] Havlíček V, Córcoles AD, Temme K, Harrow AW, Kandala A, Chow JM,
    Gambetta JM. *Supervised learning with quantum-enhanced feature spaces*.
    Nature. 2019 Mar;567(7747):209–212.
    `arXiv:1804.11326 <https://arxiv.org/abs/1804.11326>`_

    Parameters:
        training_size : Number of training samples per class.
        test_size :  Number of testing samples per class.
        n : Number of qubits (dimension of the feature space).
        gap : Separation gap :math:`\Delta` used when ``labelling_method="expectation"``.
            Default is 0.
        plot_data : If True, plots the sampled data (disabled automatically if
            ``n > 3``). Default is False.
        one_hot : If True, returns labels in one-hot format. Default is True.
        include_sample_total : If True, the function also returns the total number
            of accepted samples. Default is False.
        entanglement : Determines which second-order terms :math:`Z_i Z_j` appear in
            :math:`U_{\Phi(\vec{x})}`. The options are:

                * ``"linear"``: Includes terms :math:`Z_i Z_{i+1}`.
                * ``"circular"``: Includes ``"linear"`` terms plus :math:`Z_{n-1}Z_0`.
                * ``"full"``: Includes all pairwise terms :math:`Z_i Z_j`.

            Default is ``"full"``.
        sampling_method: The method used to generate uniform samples :math:`\vec{x}`.
            Choices are:

                * ``"grid"``: Chooses points from a uniform grid (supported only if ``n <= 3``)
                * ``"hypercube"``: Uses a variant of Latin Hypercube sampling for stratification
                * ``"sobol"``: Uses Sobol sequences

            Default is ``"grid"``.
        divisions : Must be specified if ``sampling_method="hypercube"``. This parameter
            determines the number of stratifications along each dimension. Recommended
            to be chosen close to ``training_size``.
        labelling_method : Method for assigning labels. The options are:

                * ``"expectation"``: Uses the expectation value of the observable.
                * ``"measurement"``: Performs a measurement in the computational basis.

            Default is ``"expectation"``.
        class_labels : Custom labels for the two classes when one-hot is not enabled.
            If not provided, the labels default to ``-1`` and ``+1``

    Returns:
        Tuple
        containing the following:

        * **training_features** : ``np.ndarray``
        * **training_labels** : ``np.ndarray``
        * **testing_features** : ``np.ndarray``
        * **testing_labels** : ``np.ndarray``

        If ``include_sample_total=True``, a fifth element (``np.ndarray``) is included
        that specifies the total number of accepted samples.
    """

    # Default Value
    if class_labels is None:
        class_labels = [0, 1]

    # Errors
    if training_size < 0:
        raise ValueError("Training size can't be less than 0")
    if test_size < 0:
        raise ValueError("Test size can't be less than 0")
    if n <= 0:
        raise ValueError("Number of qubits can't be less than 1")
    if gap < 0 and labelling_method == "expectation":
        raise ValueError("Gap can't be less than 0")
    if entanglement not in {"linear", "circular", "full"}:
        raise ValueError("Invalid entanglement type. Must be 'linear', 'circular', or 'full'.")
    if sampling_method not in {"grid", "hypercube", "sobol"}:
        raise ValueError("Invalid sampling method. Must be 'grid', 'hypercube', or 'sobol'.")
    if divisions == 0 and sampling_method == "hypercube":
        raise ValueError("Divisions must be set for 'hypercube' sampling.")
    if labelling_method not in {"expectation", "measurement"}:
        raise ValueError("Invalid labelling method. Must be 'expectation' or 'measurement'.")
    if n > 3 and sampling_method == "grid":
        raise ValueError("Grid sampling is unsupported for n > 3.")

    # Warnings
    if n > 3 and plot_data:
        warnings.warn(
            "Plotting for n > 3 is unsupported. Disabling plot_data.",
            UserWarning,
        )
        plot_data = False

    if sampling_method == "grid" and (training_size + test_size) > 4000:
        warnings.warn(
            """Grid Sampling for large number of samples is not recommended
            and can lead to samples repeating in the training and testing sets""",
            UserWarning,
        )

    # Initial State
    dims = 2**n
    psi_0 = np.ones(dims) / np.sqrt(dims)

    # n-qubit Hadamard
    h_n = _n_hadamard(n)

    # Single qubit Z gates
    z_diags = np.array([np.diag(_i_z(i, n)).reshape((1, -1)) for i in range(n)])

    # Precompute ZZ Entanglements
    zz_diags = {}
    if entanglement == "full":
        for i, j in it.combinations(range(n), 2):
            zz_diags[(i, j)] = z_diags[i] * z_diags[j]
    else:
        for i in range(n - 1):
            zz_diags[(i, i + 1)] = z_diags[i] * z_diags[i + 1]
        if entanglement == "circular":
            zz_diags[(n - 1, 0)] = z_diags[n - 1] * z_diags[0]

    # n-qubit Z gate: notice that h_n[0,:] has the same elements as diagonal of z_n
    z_n = _n_z(h_n)

    # V change of basis: Eigenbasis of a random hermitian will be a random unitary
    v = _random_unitary(dims)

    # Observable for labelling boundary
    mat_o = v.conj().T @ z_n @ v

    n_samples = training_size + test_size

    # Labelling Methods
    if labelling_method == "expectation":

        def _lab_fn(psi_state):
            return _exp_label(psi_state, gap, mat_o)

    else:
        eig = np.linalg.eigh(mat_o)

        def _lab_fn(psi_state):
            return _measure(psi_state, eig)

    # Sampling Methods
    if sampling_method == "grid":
        a_features, b_features = _grid_sampling(
            n, n_samples, z_diags, zz_diags, psi_0, h_n, _lab_fn
        )
    else:
        if sampling_method == "hypercube":

            def _samp_fn(a, b):
                return _modified_lhc(a, b, divisions)

        else:

            def _samp_fn(a, b):
                return _sobol_sampling(a, b)

        a_features, b_features = _loop_sampling(
            n,
            n_samples,
            z_diags,
            zz_diags,
            psi_0,
            h_n,
            _lab_fn,
            _samp_fn,
            sampling_method,
        )

    if plot_data:
        _plot_ad_hoc_data(a_features, b_features, training_size)

    x_train = np.concatenate((a_features[:training_size], b_features[:training_size]), axis=0)
    x_test = np.concatenate((a_features[training_size:], b_features[training_size:]), axis=0)
    if one_hot:
        y_train = np.array([[1, 0]] * training_size + [[0, 1]] * training_size)
        y_test = np.array([[1, 0]] * test_size + [[0, 1]] * test_size)
    else:
        y_train = np.array([class_labels[0]] * training_size + [class_labels[1]] * training_size)
        y_test = np.array([class_labels[0]] * test_size + [class_labels[1]] * test_size)

    if include_sample_total:
        samples = np.array([n_samples * 2])
        return (x_train, y_train, x_test, y_test, samples)

    return (x_train, y_train, x_test, y_test)


@optionals.HAS_MATPLOTLIB.require_in_call
def _plot_ad_hoc_data(a_features: np.ndarray, b_features: np.ndarray, training_size: int) -> None:
    """Plot the ad hoc dataset.

    Args:
        a_features (np.ndarray): Class-A feature vectors.
        b_features (np.ndarray): Class-B feature vectors.
        training_size (int): Number of training samples to plot.
    """
    import matplotlib.pyplot as plt

    fig = plt.figure()
    projection = "3d" if a_features.shape[1] == 3 else None
    ax1 = fig.add_subplot(1, 1, 1, projection=projection)
    ax1.scatter(*a_features[:training_size].T)
    ax1.scatter(*b_features[:training_size].T)
    ax1.set_title("Ad-hoc Data")
    plt.show()


def _n_hadamard(n: int) -> np.ndarray:
    """Generate an n-qubit Hadamard matrix.

    Args:
        n (int): Number of qubits.

    Returns:
        np.ndarray: The n-qubit Hadamard matrix.
    """
    base = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
    result = np.eye(1)
    expo = n

    while expo > 0:
        if expo % 2 == 1:
            result = np.kron(result, base)
        base = np.kron(base, base)
        expo //= 2

    return result


def _i_z(i: int, n: int) -> np.ndarray:
    """Create the i-th single-qubit Z gate in an n-qubit system.

    Args:
        i (int): Index of the qubit.
        n (int): Total number of qubits.

    Returns:
        np.ndarray: The Z gate acting on the i-th qubit.
    """
    z = np.diag([1, -1])
    i_1 = np.eye(2**i)
    i_2 = np.eye(2 ** (n - i - 1))

    result = np.kron(i_1, z)
    result = np.kron(result, i_2)

    return result


def _n_z(h_n: np.ndarray) -> np.ndarray:
    """Generate an n-qubit Z gate from the n-qubit Hadamard matrix.

    Args:
        h_n (np.ndarray): n-qubit Hadamard matrix.

    Returns:
        np.ndarray: The n-qubit Z gate.
    """
    res = np.diag(h_n)
    res = np.sign(res)
    res = np.diag(res)
    return res


def _modified_lhc(n: int, n_samples: int, n_div: int) -> np.ndarray:
    """Generate samples using modified Latin Hypercube Sampling.

    Args:
        n (int): Dimensionality of the data.
        n_samples (int): Number of samples to generate.
        n_div (int): Number of divisions for stratified sampling.

    Returns:
        np.ndarray: Generated samples.
    """
    samples = np.empty((n_samples, n), dtype=float)
    bin_size = 2 * np.pi / n_div
    n_passes = (n_samples + n_div - 1) // n_div

    all_bins = np.tile(np.arange(n_div), n_passes)

    for dim in range(n):
        algorithm_globals.random.shuffle(all_bins)
        chosen_bins = all_bins[:n_samples]
        offsets = algorithm_globals.random.random(n_samples)
        samples[:, dim] = (chosen_bins + offsets) * bin_size

    return samples


def _sobol_sampling(n: int, n_samples: int) -> np.ndarray:
    """Generate samples using Sobol sequence sampling.

    Args:
        n (int): Dimensionality of the data.
        n_samples (int): Number of samples to generate.

    Returns:
        np.ndarray: Generated samples scaled to the interval [0, 2π].
    """
    sampler = Sobol(d=n, scramble=True)
    p = 2 * np.pi * sampler.random(n_samples)
    return p


def _phi_i(x_vecs: np.ndarray, i: int) -> np.ndarray:
    """Compute the φ_i term for a given dimension.

    Args:
        x_vecs (np.ndarray): Input sample vectors.
        i (int): Dimension index.

    Returns:
        np.ndarray: Computed φ_i values.
    """
    return x_vecs[:, i].reshape((-1, 1))


def _phi_ij(x_vecs: np.ndarray, i: int, j: int) -> np.ndarray:
    """Compute the φ_ij term for given dimensions.

    Args:
        x_vecs (np.ndarray): Input sample vectors.
        i (int): First dimension index.
        j (int): Second dimension index.

    Returns:
        np.ndarray: Computed φ_ij values.
    """
    return ((np.pi - x_vecs[:, i]) * (np.pi - x_vecs[:, j])).reshape((-1, 1))


def _random_unitary(dims):
    a = np.array(
        algorithm_globals.random.random((dims, dims))
        + 1j * algorithm_globals.random.random((dims, dims))
    )
    herm = a.conj().T @ a
    eigvals, eigvecs = np.linalg.eig(herm)
    idx = eigvals.argsort()[::-1]
    v = eigvecs[:, idx]
    return v


def _loop_sampling(n, n_samples, z_diags, zz_diags, psi_0, h_n, lab_fn, samp_fn, sampling_method):
    """
    Loop-based sampling routine to allocate feature vectors into two classes.

    Args:
        n (int): Number of qubits (feature dimension).
        n_samples (int): Number of samples needed per class.
        z_diags (np.ndarray): Array of single-qubit Z diagonal elements.
        zz_diags (dict): Dictionary of ZZ-diagonal elements keyed by qubit
            pairs.
        O (np.ndarray): Observable for label determination.
        psi_0 (np.ndarray): Initial state vector.
        h_n (np.ndarray): n-qubit Hadamard matrix.
        lab_fn (Callable): Labeling function (either expectation-based or
            measurement-based).
        samp_fn (Callable): Sampling function that generates feature vectors.
        sampling_method (str): String indicating which sampling method is used
            ("grid", "hypercube", or "sobol").

    Returns:
        Tuple[np.ndarray, np.ndarray]:
            Two arrays of shape `(n_samples, n)`, each containing the sampled
            feature vectors belonging to class A and class B, respectively.
    """
    a_features = np.empty((n_samples, n), dtype=float)
    b_features = np.empty((n_samples, n), dtype=float)

    dims = 2**n
    a_cur, b_cur = 0, 0
    a_needed, b_needed = n_samples, n_samples

    while a_needed > 0 or b_needed > 0:
        n_pass = a_needed + b_needed

        # Sobol works better with a 2^n just above n_pass
        if sampling_method == "sobol":
            n_pass = 2 ** ((n_pass - 1).bit_length())

        # Stratified Sampling for x vector
        x_vecs = samp_fn(n, n_pass)

        pre_exp = np.zeros((n_pass, dims))

        # First Order Terms
        for i in range(n):
            pre_exp += _phi_i(x_vecs, i) * z_diags[i]

        # Second Order Terms
        for i, j in zz_diags.keys():
            pre_exp += _phi_ij(x_vecs, i, j) * zz_diags[(i, j)]

        # Since pre_exp is purely diagonal, exp(A) = diag(exp(Aii))
        post_exp = np.exp(1j * pre_exp)
        uphi = np.zeros((n_pass, dims, dims), dtype=post_exp.dtype)
        cols = range(dims)
        uphi[:, cols, cols] = post_exp[:, cols]

        psi = (uphi @ h_n @ uphi @ psi_0).reshape((-1, dims, 1))

        # Labelling
        raw_labels = lab_fn(psi)

        if a_needed > 0:
            a_indx = raw_labels == 1
            a_count = min(int(np.sum(a_indx)), a_needed)
            a_features[a_cur : a_cur + a_count] = x_vecs[a_indx][:a_count]
            a_cur += a_count
            a_needed -= a_count

        if b_needed > 0:
            b_indx = raw_labels == -1
            b_count = min(int(np.sum(b_indx)), b_needed)
            b_features[b_cur : b_cur + b_count] = x_vecs[b_indx][:b_count]
            b_cur += b_count
            b_needed -= b_count

    return a_features, b_features


def _exp_label(psi, gap, mat_o):
    """
    Compute labels by comparing the expectation value of an observable to a gap.

    Args:
        psi (np.ndarray): Array of shape `(num_samples, dim, 1)` containing
            the statevectors for each sample.
        gap (float): Separation gap (Δ). If the absolute expectation exceeds
            this, the sample is labeled ±1 based on the sign.
        O (np.ndarray): Observable used for label determination.

    Returns:
        np.ndarray: Labels for each sample. Values will be -1, 0, or +1, where
        0 indicates an expectation value within the gap zone (not exceeding ±gap).
    """
    psi_dag = np.transpose(psi.conj(), (0, 2, 1))
    exp_val = np.real(psi_dag @ mat_o @ psi).flatten()
    labels = (np.abs(exp_val) > gap) * (np.sign(exp_val))
    return labels


def _measure(psi, eig):
    """
    Compute labels by simulating a measurement of the observable on each state.

    The eigen-decomposition of O is used as the measurement basis. Each state
    is projected onto one of the eigenvectors, and labels are set to the
    corresponding eigenvalue.

    Args:
        psi (np.ndarray): Array of shape `(num_samples, dim, 1)` containing
            the statevectors for each sample.
        eig (np.ndarray): Eigenvalues of Observable to be 'measured'

    Returns:
        np.ndarray: Labels for each sample, set to one of the eigenvalues
        of the observable O.
    """
    eigenvalues, eigenvectors = eig
    eigshape = eigenvectors.shape
    new_psi = eigenvectors.T.conj().reshape((1, eigshape[1], eigshape[0])) @ psi

    probab = np.abs(new_psi) ** 2
    toss = algorithm_globals.random.random((psi.shape[0], 1))
    cum_probab = np.cumsum(probab, axis=1).reshape(psi.shape[0], -1)
    collapsed = (cum_probab >= toss).argmax(axis=-1, keepdims=True)
    labels = eigenvalues[collapsed.flatten()]

    return np.sign(labels)


def _grid_sampling(n, n_samples, z_diags, zz_diags, psi_0, h_n, lab_fn):
    """
    Generate feature vectors from a uniform grid (only supported for `n <= 3`)
    and assign labels using the specified labeling function.

    Args:
        n (int): Number of qubits (dimension).
        n_samples (int): Number of samples needed per class.
        z_diags (np.ndarray): Array of single-qubit Z diagonal elements.
        zz_diags (dict): Dictionary of ZZ-diagonal elements keyed by qubit pairs.
        psi_0 (np.ndarray): Initial state vector.
        h_n (np.ndarray): n-qubit Hadamard matrix.
        lab_fn (Callable): Labeling function (either expectation-based or
            measurement-based).

    Returns:
        Tuple[np.ndarray, np.ndarray]:
            Two arrays of shape `(n_samples, n)`, each containing the sampled
            feature vectors belonging to class A and class B, respectively.
            This code is incomplete and references variables not defined above,
            so the returned arrays are empty placeholders by default.
    """

    count = 1
    if n == 1:
        count = 5000
    elif n == 2:
        count = 100
    elif n == 3:
        count = 20

    xvals = np.linspace(0, 2 * np.pi, count, endpoint=False)
    grid_labels = []

    # Loop through uniform grid
    for x in it.product(*[xvals] * n):
        x_arr = np.array(x)
        pre_exp = 0
        for i in range(n):
            pre_exp += x_arr[i] * z_diags[i]
        for i, j in zz_diags.keys():
            pre_exp += ((np.pi - x_arr[i]) * (np.pi - x_arr[j])) * zz_diags[(i, j)]

        uphi = np.diag(np.exp(1j * pre_exp.flatten()))
        psi = uphi @ h_n @ uphi @ psi_0
        label = lab_fn(psi.reshape((1, -1, 1)))

        grid_labels.append(label)

    grid_labels = np.array(grid_labels).reshape(*[count] * n)

    count = grid_labels.shape[0]
    a_features, b_features = [], []

    while len(a_features) < n_samples:
        draws = tuple(algorithm_globals.random.choice(count) for _ in range(n))
        if grid_labels[draws] == 1:
            a_features.append([xvals[d] for d in draws])

    while len(b_features) < n_samples:
        draws = tuple(algorithm_globals.random.choice(count) for _ in range(n))
        if grid_labels[draws] == -1:
            b_features.append([xvals[d] for d in draws])

    return np.array(a_features), np.array(b_features)

In [ ]:
# This code is part of a Qiskit project.
#
# (C) Copyright IBM 2020, 2025.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

""" Test Ad Hoc Data """

from test import QiskitMachineLearningTestCase

import unittest
import json
import numpy as np
from ddt import ddt, unpack, idata

from qiskit_machine_learning.utils import algorithm_globals
from qiskit_machine_learning.datasets import ad_hoc_data


@ddt
class TestAdHocData(QiskitMachineLearningTestCase):
    """Ad Hoc Data tests."""

    @idata(
        ([2], [3]),
    )
    @unpack
    def test_ad_hoc_data(self, num_features):
        """Ad Hoc Data test."""
        training_features, training_labels, _, test_labels = ad_hoc_data(
            training_size=20,
            test_size=10,
            n=num_features,
            gap=0.3,
            plot_data=False,
            one_hot=False,
        )
        np.testing.assert_array_equal(training_features.shape, (40, num_features))
        np.testing.assert_array_equal(training_labels.shape, (40,))
        np.testing.assert_array_almost_equal(test_labels, np.array([0] * 10 + [1] * 10))

        # Now one_hot=True
        _, _, _, test_labels_oh = ad_hoc_data(
            training_size=20,
            test_size=10,
            n=num_features,
            gap=0.3,
            plot_data=False,
            one_hot=True,
        )
        np.testing.assert_array_equal(test_labels_oh.shape, (20, 2))
        np.testing.assert_array_equal(test_labels_oh, np.array([[1, 0]] * 10 + [[0, 1]] * 10))

    def test_ref_data(self):
        """Tests ad hoc against known reference data"""
        input_file = self.get_resource_path("ad_hoc_ref.json", "datasets")
        with open(input_file, encoding="utf8") as file:
            ref_data = json.load(file)

        for seed in ref_data:
            algorithm_globals.random_seed = int(seed)
            (
                training_features,
                training_labels,
                test_features,
                test_labels,
            ) = ad_hoc_data(
                training_size=20,
                test_size=5,
                n=2,
                gap=0.3,
                plot_data=False,
                one_hot=False,
            )
            with self.subTest("Test training_features"):
                np.testing.assert_almost_equal(
                    ref_data[seed]["training_features"],
                    training_features,
                )
            with self.subTest("Test training_labels"):
                np.testing.assert_almost_equal(
                    ref_data[seed]["training_labels"],
                    training_labels,
                )
            with self.subTest("Test test_features"):
                np.testing.assert_almost_equal(
                    ref_data[seed]["test_features"],
                    test_features,
                )
            with self.subTest("Test test_labels"):
                np.testing.assert_almost_equal(
                    ref_data[seed]["test_labels"],
                    test_labels,
                )

    def test_entanglement_linear(self):
        """Test linear entanglement."""
        (
            training_features,
            training_labels,
            test_features,
            test_labels,
        ) = ad_hoc_data(
            training_size=10,
            test_size=5,
            n=2,
            plot_data=False,
            one_hot=False,
            entanglement="linear",
        )
        self.assertEqual(training_features.shape, (20, 2))
        self.assertEqual(training_labels.shape, (20,))
        self.assertEqual(test_features.shape, (10, 2))
        self.assertEqual(test_labels.shape, (10,))

    def test_entanglement_circular(self):
        """Test circular entanglement."""
        (
            training_features,
            training_labels,
            test_features,
            test_labels,
        ) = ad_hoc_data(
            training_size=10,
            test_size=5,
            n=2,
            plot_data=False,
            one_hot=False,
            entanglement="circular",
        )
        self.assertEqual(training_features.shape, (20, 2))
        self.assertEqual(training_labels.shape, (20,))
        self.assertEqual(test_features.shape, (10, 2))
        self.assertEqual(test_labels.shape, (10,))

    def test_entanglement_full(self):
        """Test full entanglement."""
        (
            training_features,
            training_labels,
            test_features,
            test_labels,
        ) = ad_hoc_data(
            training_size=10,
            test_size=5,
            n=2,
            plot_data=False,
            one_hot=False,
            entanglement="full",
        )
        self.assertEqual(training_features.shape, (20, 2))
        self.assertEqual(training_labels.shape, (20,))
        self.assertEqual(test_features.shape, (10, 2))
        self.assertEqual(test_labels.shape, (10,))

    def test_sampling_grid(self):
        """Test grid sampling method."""
        (
            training_features,
            training_labels,
            test_features,
            test_labels,
        ) = ad_hoc_data(
            training_size=10,
            test_size=5,
            n=2,
            sampling_method="grid",
            plot_data=False,
            one_hot=False,
        )
        self.assertEqual(training_features.shape, (20, 2))
        self.assertEqual(training_labels.shape, (20,))
        self.assertEqual(test_features.shape, (10, 2))
        self.assertEqual(test_labels.shape, (10,))

    def test_sampling_sobol(self):
        """Test Sobol sampling method."""
        (
            training_features,
            training_labels,
            test_features,
            test_labels,
        ) = ad_hoc_data(
            training_size=10,
            test_size=5,
            n=2,
            sampling_method="sobol",
            plot_data=False,
            one_hot=False,
        )
        self.assertEqual(training_features.shape, (20, 2))
        self.assertEqual(training_labels.shape, (20,))
        self.assertEqual(test_features.shape, (10, 2))
        self.assertEqual(test_labels.shape, (10,))

    def test_sampling_hypercube(self):
        """Test hypercube sampling with divisions parameter."""
        (
            training_features,
            training_labels,
            test_features,
            test_labels,
        ) = ad_hoc_data(
            training_size=10,
            test_size=5,
            n=2,
            sampling_method="hypercube",
            divisions=10,
            plot_data=False,
            one_hot=False,
        )
        self.assertEqual(training_features.shape, (20, 2))
        self.assertEqual(training_labels.shape, (20,))
        self.assertEqual(test_features.shape, (10, 2))
        self.assertEqual(test_labels.shape, (10,))

    def test_labelling_expectation(self):
        """Test expectation labelling method."""
        (
            training_features,
            training_labels,
            test_features,
            test_labels,
        ) = ad_hoc_data(
            training_size=10,
            test_size=5,
            n=2,
            plot_data=False,
            one_hot=False,
            labelling_method="expectation",
        )
        self.assertEqual(training_features.shape, (20, 2))
        self.assertEqual(training_labels.shape, (20,))
        self.assertEqual(test_features.shape, (10, 2))
        self.assertEqual(test_labels.shape, (10,))

    def test_labelling_measurement(self):
        """Test measurement labelling method."""
        (
            training_features,
            training_labels,
            test_features,
            test_labels,
        ) = ad_hoc_data(
            training_size=10,
            test_size=5,
            n=2,
            plot_data=False,
            one_hot=False,
            labelling_method="measurement",
        )
        self.assertEqual(training_features.shape, (20, 2))
        self.assertEqual(training_labels.shape, (20,))
        self.assertEqual(test_features.shape, (10, 2))
        self.assertEqual(test_labels.shape, (10,))

    def test_custom_class_labels(self):
        """Test custom class labels."""
        custom_labels = ["Class1", "Class2"]
        (
            _,
            training_labels,
            _,
            _,
        ) = ad_hoc_data(
            training_size=10,
            test_size=5,
            n=2,
            plot_data=False,
            one_hot=False,
            class_labels=custom_labels,
        )

        unique_labels = np.unique(training_labels)
        self.assertEqual(len(unique_labels), 2)
        for label in custom_labels:
            self.assertIn(label, unique_labels)

        # Test with one_hot=True
        (
            _,
            training_labels_onehot,
            _,
            test_labels_onehot,
        ) = ad_hoc_data(
            training_size=10,
            test_size=5,
            n=2,
            plot_data=False,
            one_hot=True,
            class_labels=custom_labels,
        )

        self.assertEqual(training_labels_onehot.shape, (20, 2))
        self.assertEqual(test_labels_onehot.shape, (10, 2))

    def test_include_sample_total(self):
        """Test include_sample_total parameter returns 5-tuple."""
        result = ad_hoc_data(
            training_size=10,
            test_size=5,
            n=2,
            plot_data=False,
            one_hot=False,
            include_sample_total=True,
        )
        self.assertEqual(len(result), 5)
        np.testing.assert_array_equal(result[4], np.array([30]))

    def test_higher_qubits(self):
        """Test with dimensions higher than 3 (n=4)."""
        (
            training_features,
            _,
            test_features,
            _,
        ) = ad_hoc_data(
            training_size=5,
            test_size=3,
            n=4,
            plot_data=False,
            one_hot=False,
            sampling_method="sobol",
        )
        self.assertEqual(training_features.shape, (10, 4))
        self.assertEqual(test_features.shape, (6, 4))

    def test_error_cases(self):
        """Test error cases in the new implementation."""

        # Test negative training_size
        with self.assertRaises(ValueError):
            ad_hoc_data(training_size=-1, test_size=5, n=2)

        # Test negative test_size
        with self.assertRaises(ValueError):
            ad_hoc_data(training_size=5, test_size=-1, n=2)

        # Test invalid n
        with self.assertRaises(ValueError):
            ad_hoc_data(training_size=5, test_size=5, n=0)

        # Test negative gap with expectation labelling
        with self.assertRaises(ValueError):
            ad_hoc_data(
                training_size=5,
                test_size=5,
                n=2,
                gap=-1,
                labelling_method="expectation",
            )

        # Test invalid entanglement
        with self.assertRaises(ValueError):
            ad_hoc_data(
                training_size=5,
                test_size=5,
                n=2,
                entanglement="invalid",
            )

        # Test invalid sampling method
        with self.assertRaises(ValueError):
            ad_hoc_data(
                training_size=5,
                test_size=5,
                n=2,
                sampling_method="invalid",
            )

        # Test hypercube without divisions
        with self.assertRaises(ValueError):
            ad_hoc_data(
                training_size=5,
                test_size=5,
                n=2,
                sampling_method="hypercube",
            )

        # Test invalid labelling method
        with self.assertRaises(ValueError):
            ad_hoc_data(
                training_size=5,
                test_size=5,
                n=2,
                labelling_method="invalid",
            )

        # Test grid sampling with n > 3
        with self.assertRaises(ValueError):
            ad_hoc_data(
                training_size=5,
                test_size=5,
                n=4,
                sampling_method="grid",
            )

    def test_hypercube_sampling_linear_entanglement(self):
        """Test hypercube sampling and linear entanglement."""
        (
            training_features,
            _,
            test_features,
            _,
        ) = ad_hoc_data(
            training_size=10,
            test_size=5,
            n=2,
            plot_data=False,
            one_hot=False,
            sampling_method="hypercube",
            divisions=12,
            entanglement="linear",
        )
        self.assertEqual(training_features.shape, (20, 2))
        self.assertEqual(test_features.shape, (10, 2))

    def test_custom_labels_circular_entanglement(self):
        """Test custom labels with circular entanglement."""
        custom_labels = ["Yes", "No"]
        (
            training_features,
            training_labels,
            test_features,
            _,
        ) = ad_hoc_data(
            training_size=8,
            test_size=4,
            n=3,
            plot_data=False,
            one_hot=False,
            entanglement="circular",
            class_labels=custom_labels,
        )
        self.assertEqual(training_features.shape, (16, 3))
        self.assertEqual(test_features.shape, (8, 3))
        unique_labels = np.unique(training_labels)
        self.assertIn("Yes", unique_labels)
        self.assertIn("No", unique_labels)

    def test_measurement_sobol_sampling(self):
        """Test custom labels with circular entanglement."""
        (
            training_features,
            _,
            test_features,
            _,
        ) = ad_hoc_data(
            training_size=8,
            test_size=4,
            n=3,
            plot_data=False,
            one_hot=False,
            labelling_method="measurement",
            sampling_method="sobol",
        )
        self.assertEqual(training_features.shape, (16, 3))
        self.assertEqual(test_features.shape, (8, 3))

    def test_expectation_labelling_with_gap(self):
        """Test expectation labelling with a non-zero gap."""
        (
            training_features,
            _,
            test_features,
            _,
        ) = ad_hoc_data(
            training_size=10,
            test_size=5,
            n=2,
            gap=0.5,
            plot_data=False,
            one_hot=False,
            labelling_method="expectation",
        )
        self.assertEqual(training_features.shape, (20, 2))
        self.assertEqual(test_features.shape, (10, 2))


if __name__ == "__main__":
    unittest.main()

In [ ]:
# This code is part of a Qiskit project.
#
# (C) Copyright IBM 2023, 2024.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

"""The QNN circuit."""
from __future__ import annotations
from typing import List

from qiskit.circuit import QuantumRegister, QuantumCircuit
from qiskit.circuit.parametertable import ParameterView
from qiskit.circuit.library import BlueprintCircuit

from qiskit_machine_learning import QiskitMachineLearningError

from ...utils import derive_num_qubits_feature_map_ansatz


class QNNCircuit(BlueprintCircuit):
    """
    The QNN circuit is a blueprint circuit that wraps feature map and ansatz circuits.
    It can be used to simplify the composition of these two.

    If only the number of qubits is provided the :class:`~qiskit.circuit.library.RealAmplitudes`
    ansatz and the :class:`~qiskit.circuit.library.ZZFeatureMap` feature map are used. If the
    number of qubits is 1 the :class:`~qiskit.circuit.library.ZFeatureMap` is used. If only a
    feature map is provided, the :class:`~qiskit.circuit.library.RealAmplitudes` ansatz with the
    corresponding number of qubits is used. If only an ansatz is provided the
    :class:`~qiskit.circuit.library.ZZFeatureMap` with the corresponding number of qubits is used.

    At least one parameter has to be provided. If a feature map and an ansatz is provided, the
    number of qubits must be the same.

    In case number of qubits is provided along with either a feature map, an ansatz or both, a
    potential mismatch between the three inputs with respect to the number of qubits is resolved by
    constructing the :class:`~qiskit_machine_learning.circuit.library.QNNCircuit` with the given
    number of qubits. If one of the :class:`~qiskit_machine_learning.circuit.library.QNNCircuit`
    properties is set after the class construction, the circuit is adjusted  to incorporate the
    changes. This means, a new valid configuration that considers the latest property update will be
    derived. This ensures that the classes properties are consistent at all times.

    Example:

    .. code-block:: python

        from qiskit_machine_learning.circuit.library import QNNCircuit
        qnn_qc = QNNCircuit(2)
        print(qnn_qc)
        # prints:
        #      ┌──────────────────────────┐»
        # q_0: ┤0                         ├»
        #      │  ZZFeatureMap(x[0],x[1]) │»
        # q_1: ┤1                         ├»
        #      └──────────────────────────┘»
        # «     ┌──────────────────────────────────────────────────────────┐
        # «q_0: ┤0                                                         ├
        # «     │  RealAmplitudes(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7]) │
        # «q_1: ┤1                                                         ├
        # «     └──────────────────────────────────────────────────────────┘

        print(qnn_qc.num_qubits)
        # prints: 2

        print(qnn_qc.input_parameters)
        # prints: ParameterView([ParameterVectorElement(x[0]), ParameterVectorElement(x[1])])

        print(qnn_qc.weight_parameters)
        # prints: ParameterView([ParameterVectorElement(θ[0]), ParameterVectorElement(θ[1]),
        #         ParameterVectorElement(θ[2]), ParameterVectorElement(θ[3]),
        #         ParameterVectorElement(θ[4]), ParameterVectorElement(θ[5]),
        #         ParameterVectorElement(θ[6]), ParameterVectorElement(θ[7])])
    """

    def __init__(
        self,
        num_qubits: int | None = None,
        feature_map: QuantumCircuit | None = None,
        ansatz: QuantumCircuit | None = None,
    ) -> None:
        """
        Although all parameters default to None at least one parameter must be provided, to determine
        the number of qubits from it, when the instance is created.

        If more than one parameter is passed:

        1) If num_qubits is provided the feature map and/or ansatz supplied will be overridden to
        circuits with num_qubits, as long as the respective circuit supports updating its number of
        qubits.

        2) If num_qubits is not provided the feature_map and ansatz must be set to the same number
        of qubits.

        Args:
            num_qubits:  Number of qubits, a positive integer. Optional if feature_map or ansatz is
                         provided, otherwise required. If not provided num_qubits defaults from the
                         sizes of feature_map and ansatz.
            feature_map: A feature map. Optional if num_qubits or ansatz is provided, otherwise
                         required. If not provided defaults to
                         :class:`~qiskit.circuit.library.ZZFeatureMap` or
                         :class:`~qiskit.circuit.library.ZFeatureMap` if num_qubits is determined
                         to be 1.
            ansatz:      An ansatz. Optional if num_qubits or feature_map is provided, otherwise
                         required. If not provided defaults to
                         :class:`~qiskit.circuit.library.RealAmplitudes`.

        Returns:
            The composed feature map and ansatz circuit.

        Raises:
            QiskitMachineLearningError: If a valid number of qubits cannot be derived from the \
            provided input arguments.
        """

        super().__init__()
        self._feature_map = feature_map
        self._ansatz = ansatz
        # Check if circuit is constructed with valid configuration and set properties accordingly.
        self.num_qubits, self._feature_map, self._ansatz = derive_num_qubits_feature_map_ansatz(
            num_qubits, feature_map, ansatz
        )

    def _build(self):
        super()._build()
        self.compose(self.feature_map, inplace=True)
        self.compose(self.ansatz, inplace=True)

    def _check_configuration(self, raise_on_failure=True):
        try:
            self.num_qubits, self.feature_map, self.ansatz = derive_num_qubits_feature_map_ansatz(
                self.num_qubits, self.feature_map, self.ansatz
            )
        except QiskitMachineLearningError as qml_ex:
            if raise_on_failure:
                raise qml_ex

    @property
    def num_qubits(self) -> int:
        """Returns the number of qubits in this circuit.

        Returns:
            The number of qubits.
        """
        return super().num_qubits

    @num_qubits.setter
    def num_qubits(self, num_qubits: int) -> None:
        """Set the number of qubits. If num_qubits is set
        the feature map and ansatz are adjusted to circuits with num_qubits qubits.

        Args:
            num_qubits:  The number of qubits, a positive integer.
        """
        if self.num_qubits != num_qubits:
            # invalidate the circuit
            self._invalidate()
            self.qregs: List[QuantumRegister] = []
            if num_qubits is not None and num_qubits > 0:
                self.qregs = [QuantumRegister(num_qubits, name="q")]
                (
                    self.num_qubits,
                    self._feature_map,
                    self._ansatz,
                ) = derive_num_qubits_feature_map_ansatz(
                    num_qubits, self._feature_map, self._ansatz
                )

    @property
    def feature_map(self) -> QuantumCircuit:
        """Returns feature_map.

        Returns:
            The feature map.
        """
        return self._feature_map

    @feature_map.setter
    def feature_map(self, feature_map: QuantumCircuit) -> None:
        """Set the feature map. If the feature map is updated the ``QNNCircuit`` is adjusted
        according to the feature map being passed. This includes:
        1) The num_qubits is adjusted to the feature map number of qubits.
        2) The ansatz is adjusted to a circuit with the feature_map number of qubits.

        Args:
            feature_map: The feature map.
        """
        if self.feature_map != feature_map:
            # invalidate the circuit
            self._invalidate()
            self.num_qubits = feature_map.num_qubits
            self.num_qubits, self._feature_map, self._ansatz = derive_num_qubits_feature_map_ansatz(
                self.num_qubits, feature_map, self.ansatz
            )

    @property
    def ansatz(self) -> QuantumCircuit:
        """Returns ansatz.

        Returns:
            The ansatz.
        """
        return self._ansatz

    @ansatz.setter
    def ansatz(self, ansatz: QuantumCircuit) -> None:
        """Set the ansatz. If the ansatz is updated the ``QNNCircuit`` is adapted
        according to the ansatz being passed. This includes:
        1) The num_qubits is adjusted to the ansatz number of qubits.
        2) The feature_map is adjusted to a circuit with the ansatz number of qubits.

        Args:
            ansatz: The ansatz.
        """
        if self.ansatz != ansatz:
            # invalidate the circuit
            self._invalidate()
            self.num_qubits = ansatz.num_qubits
            self.num_qubits, self._feature_map, self._ansatz = derive_num_qubits_feature_map_ansatz(
                self.num_qubits, self.feature_map, ansatz
            )

    @property
    def input_parameters(self) -> ParameterView:
        """Returns the parameters of the feature map.

        Returns:
            The parameters of the feature map.
        """
        return self._feature_map.parameters

    @property
    def num_input_parameters(self) -> int:
        """Returns the number of input parameters in the circuit.

        Returns:
            The number of input parameters.
        """
        return len(self._feature_map.parameters)

    @property
    def weight_parameters(self) -> ParameterView:
        """Returns the parameters of the ansatz. These corresponding to the trainable weights.

        Returns:
            The parameters of the ansatz.
        """
        return self._ansatz.parameters

    @property
    def num_weight_parameters(self) -> int:
        """Returns the number of weights in the circuit.

        Returns:
            The number of weights.
        """
        return len(self._ansatz.parameters)